# Recomendation system
## Content based filtering

In [1]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

In [2]:
ratings = pd.read_csv('data/ratings.csv', low_memory=False)
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [3]:
movie_metadata = pd.read_csv("data/movies_metadata.csv", low_memory=False)
movie_metadata = movie_metadata[['id', 'title']]
movie_metadata = movie_metadata.drop([19730, 29503, 35587])
movie_metadata =movie_metadata[pd.to_numeric(movie_metadata['id'], errors='coerce').notnull()]
movie_metadata=movie_metadata.rename(columns = {'id':'movieId'})
movie_metadata['movieId'] = movie_metadata['movieId'].astype(int)
movie_metadata.head()

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [4]:
movie_data = ratings.merge(movie_metadata, how='left', on='movieId')
movie_data.head()

,userId,movieId,rating,title
0,1,110,1.0,Three Colors: Red
1,1,147,4.5,The 400 Blows
2,1,858,5.0,Sleepless in Seattle
3,1,1221,5.0,NaN
4,1,1246,5.0,Rocky Balboa


In [5]:
df_movies_cnt = pd.DataFrame(ratings.groupby('movieId').size(), columns=['count'])
popular_movies = list(set(df_movies_cnt.query('count >= 150').index))
movies_filter = ratings.movieId.isin(popular_movies).values

df_users_cnt = pd.DataFrame(ratings.groupby('userId').size(),columns=['count'])
active_users = list(set(df_users_cnt.query('count >= 15').index))
users_filter = ratings.userId.isin(active_users).values

df_ratings_filtered = ratings[movies_filter & users_filter]
df_ratings_filtered.head()

In [7]:
reader = Reader()
ratings_by_users = Dataset.load_from_df(df_ratings_filtered[['userId', 'movieId', 'rating']], reader)
train_df, test_df = train_test_split(ratings_by_users, test_size=.2)
test_df

[(172366, 1, 3.0),
 (57845, 3534, 3.0),
 (157480, 63082, 3.5),
 (117395, 1594, 4.0),
 (60885, 1371, 3.0),
 (107963, 94959, 2.5),
 (218998, 3786, 2.0),
 (250382, 1059, 4.0),
 (93917, 4890, 3.0),
 (57066, 79293, 0.5),
 (59449, 562, 4.0),
 (182305, 58, 5.0),
 (160153, 52460, 2.5),
 (256688, 2098, 3.0),
 (65012, 1608, 4.0),
 (168139, 7153, 4.0),
 (167877, 50, 5.0),
 (61379, 2011, 4.0),
 (11847, 434, 3.0),
 (72545, 1282, 3.5),
 (2955, 33004, 4.0),
 (119898, 5346, 1.0),
 (251961, 2302, 4.0),
 (30323, 357, 5.0),
 (26257, 71535, 5.0),
 (126901, 1972, 0.5),
 (235287, 1777, 3.0),
 (227111, 1222, 3.5),
 (18640, 1101, 4.0),
 (263066, 2951, 4.0),
 (245152, 98154, 5.0),
 (117992, 2394, 3.0),
 (67827, 10, 3.0),
 (167988, 912, 5.0),
 (81095, 2144, 4.0),
 (117859, 4270, 3.0),
 (94779, 47937, 3.5),
 (249943, 3826, 3.5),
 (254892, 104881, 2.0),
 (42347, 1379, 3.0),
 (147954, 1968, 3.0),
 (138722, 5444, 3.0),
 (183473, 368, 3.5),
 (1261, 36529, 3.0),
 (187211, 48738, 1.0),
 (141358, 4232, 4.0),
 (233923, 

In [8]:
svd_model = SVD()
svd_model_trained = svd_model.fit(train_df)

In [11]:
def get_recommendations(user_id):
    pred_series= []
    users_ratings = ratings[ratings['userId'] == user_id]
    for movie_id, name in zip(movie_metadata.index, movie_metadata['title']):
        rating_real = ratings.query(f'movieId == {movie_id}')['rating'].values[0] if movie_id in users_ratings['movieId'].values else 0
        rating_pred = svd_model_trained.predict(user_id, movie_id, rating_real, verbose=False)
        pred_series.append([movie_id, name, rating_pred.est, rating_real])

    # print the results
    df_recommendations = pd.DataFrame(pred_series, columns=['movieId', 'title', 'predicted_rating', 'actual_rating'])
    return df_recommendations.sort_values(by='predicted_rating', ascending=False).head(10)

In [12]:
get_recommendations(57845)

,movieId,title,predicted_rating,actual_rating
110,110,Rumble in the Bronx,4.981295,0.0
527,527,Serial Mom,4.930384,4.0
2396,2396,Eight Days a Week,4.906985,5.0
3578,3578,The Brother from Another Planet,4.888135,4.0
318,318,Swimming with Sharks,4.884742,4.0
2762,2762,Hell Night,4.864216,4.5
2028,2028,Charlotte's Web,4.834317,4.0
1221,1221,Unforgiven,4.768528,0.0
4995,4995,The Deadly Mantis,4.750121,4.0
858,858,Bogus,4.739516,0.0
